*Disclaimer*: This notebook borrows from different sources including the [IBM Quantum Challenge from 2020](https://github.com/qiskit-community/IBMQuantumChallenge2020).

# Grover's Algorithm

Grover's algorithm also called quantum search algorithm allows searching unstructured data or more general, checking conditions for given data. This algorithm does not give an exponential speedup, but a quadratic one. Still, it can be used in different algorithms to speed up specific parts as classical algorithms require in the order of $N$ steps to search $N$ entries.

## Qiskit Implementation

After having seen the theoretical aspects of Grover's algorithm let us now look at its implementation based on those gates we got to know already. In the following we look at a two-qubit data register and use one additional qubit for the oracle.

Note that in many literature the basic Grover 2-qubit algorithm looks a bit different and does not require an additional ancillary qubit. We do this in purpose as we did not officially introduce [CZ gates](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CZGate.html) yet during the lecture. The big advantage of the shown approach is that you are perfectly equipped for more complex oracles as well as scaling to bigger amounts of qubits.

But first let's prepare our environment.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# importing qiskit
from qiskit import IBMQ, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

from qiskit.tools.visualization import plot_histogram

As in other examples discussed before, we again assume an equal superposition. In this example we will try to find the state $|11\rangle$. We therefore define our phase oracle to exactly *test* on this state and mark it accordingly.

In [ ]:
def phase_oracle(databits: int, oraclebits: int) -> QuantumCircuit:
    data_register = QuantumRegister(databits)
    oracle_register = QuantumRegister(oraclebits)
    circuit = QuantumCircuit(data_register, oracle_register)
    circuit.ccx(*data_register, oracle_register)
    return circuit

data_register = QuantumRegister(2, "d")
oracle_register = QuantumRegister(1, "o")
oraclecircuit = QuantumCircuit(data_register, oracle_register)
oraclecircuit = oraclecircuit.compose(phase_oracle(2, 1), range(3))
oraclecircuit.draw()

Next, we setup up a circuit to invert about the average. This is our amplitude amplification circuit that is commonly called diffusion circuit.

In [ ]:
def inversion_about_average(databits: int, oraclebits: int):
    """Apply inversion about the average step"""
    data_register = QuantumRegister(databits)
    oracle_register = QuantumRegister(oraclebits)
    inversion_circuit = QuantumCircuit(data_register, oracle_register)
    inversion_circuit.h(data_register)
    inversion_circuit.x(data_register)
    inversion_circuit.toffoli(*data_register, oracle_register)
    inversion_circuit.x(data_register)
    inversion_circuit.h(data_register)
    return inversion_circuit

In [ ]:
qAverage = QuantumCircuit(3)
qAverage = qAverage.compose(inversion_about_average(2, 1), range(3))
qAverage.draw()

Being prepared with the basic operations we require to implement Grover's algorithm, i.e. the phase oracle and diffusion operation, we can now put all the pieces together. It is important to include the creation of equal superposition at the start of the circuit and the measurement at the end.

Since we only have one solutions and four possibilities, we only need to run one iteration.

In [ ]:
databits = 2
oraclebits = 1

data_register = QuantumRegister(databits, "d")
oracle_register = QuantumRegister(oraclebits, "o")
cr = ClassicalRegister(databits, "c")

groverCircuit = QuantumCircuit(data_register, oracle_register, cr)
groverCircuit.h(data_register)

groverCircuit.x(oracle_register)
groverCircuit.h(oracle_register)

groverCircuit.barrier()
groverCircuit = groverCircuit.compose(
    phase_oracle(databits, oraclebits), qubits=range(databits+oraclebits))
groverCircuit.barrier()
groverCircuit = groverCircuit.compose(
    inversion_about_average(databits, oraclebits), qubits=range(databits+oraclebits))
groverCircuit.barrier()

groverCircuit.measure(data_register, cr)
groverCircuit.draw()

### Experiment with simulator

We now run the above circuit in our `aer_simulator`.

In [ ]:
backend = Aer.get_backend("aer_simulator")
job = backend.run(groverCircuit)
result = job.result()
plot_histogram(result.get_counts())

As we can see, the algorithm discovers our marked state.

Let's generalize the code of our diffusion operation a bit to enable application to a flexible number of qubits.

In [ ]:
def diffuser(databits: int, oraclebits: int, ancillabits: int):
    data_register = QuantumRegister(databits)
    oracle_register = QuantumRegister(oraclebits)
    ancilla_register = QuantumRegister(ancillabits)
    qc = QuantumCircuit(data_register, oracle_register, ancilla_register)
    # Apply transformation |s> -> |00..0> (H-gates)
    qc.h(data_register)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    qc.x(data_register)
    # Do multi-controlled-X gate
    if ancillabits > 0:
        mode = "basic"
    else:
        mode = "noancilla"
    qc.mcx(
        control_qubits=data_register, 
        target_qubit=oracle_register, 
        ancilla_qubits=ancilla_register, mode=mode)
    # Apply transformation |11..1> -> |00..0>
    qc.x(data_register)
    # Apply transformation |00..0> -> |s>
    qc.h(data_register)
    return qc

In [ ]:
diffuser_circuit = diffuser(2, 1, 0)
diffuser_circuit.draw()

## Number of iterations

As mentioned before the number of Grover iterations for a fully amplified solution is approximately $\sqrt{N}$. Let's look at how the amplification changes with a changing amount of iterations with an examplary database of size $N = 2^4$.

In [ ]:
from qiskit import BasicAer, execute

In [ ]:
backend = BasicAer.get_backend("qasm_simulator")
probabilities = []

database_count = 4
answer = "0111"

for x in range(15):
    data_register = QuantumRegister(database_count)
    oracle_register = QuantumRegister(1)
    ancillary = QuantumRegister(5)
    cr = ClassicalRegister(database_count)
    qc = QuantumCircuit(data_register, oracle_register, ancillary, cr)
    qc.h(data_register[:])
    qc.x(oracle_register[0])
    qc.h(oracle_register[0])

    for j in range(x):
        # for state with first qubit 0
        qc.x(data_register[0])
        qc.mcx(data_register[:], oracle_register[:], ancillary[:], mode="basic")
        qc.x(data_register[0])
        # apply diffusion
        qc.append(diffuser(database_count, 1, 5), [*data_register, *oracle_register, *ancillary])

    qc.h(oracle_register[0])
    qc.x(oracle_register[0])
    qc.measure(data_register, cr)

    # Change the endian
    qc = qc.reverse_bits()

    job = execute(qc, backend=backend, shots=1000, seed_simulator=1337)
    result = job.result()
    count = result.get_counts()
    try:
        probabilities.append(count[answer])
    except KeyError:
        probabilities.append(0)

In [ ]:
import math
def iterations(n):
    return math.pi * math.sqrt(n) / 4 - 1/2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
iteration = [i for i in range(15)]
correct = probabilities
plt.bar(iteration, correct)
plt.xlabel("# of iterations")
plt.ylabel("# of times the solution was obtained")
print(f"Expected maximum at {round(iterations(2**database_count))}")
plt.show()

# Optional Homework Exercise

## Solving a Lights Out Puzzle with Grover

Lights out is a famous puzzle game where you have given a rectangular grid of lights that are either switched on or off. When you flip one of the lights this also changes the state of lights in directly adjacent cells – top, bottom, left, and right. Your goal is to turn off all the lights from a random starting light pattern.

## Example Puzzle

<table style="float:right;display:inline">
<tr><td style="background-color:blue">&nbsp;</td><td style="background-color:blue">&nbsp;</td><td style="background-color:blue">&nbsp;</td></tr>
<tr><td style="background-color:yellow">&nbsp;</td><td style="background-color:blue">&nbsp;</td><td style="background-color:yellow">&nbsp;</td></tr>
<tr><td style="background-color:yellow">&nbsp;</td><td style="background-color:yellow">&nbsp;</td><td style="background-color:blue">&nbsp;</td></tr>
</table>

An example of the puzzle with a 3x3 grid is shown in the table on the right. The squares are labelled from `0` to `8`. We can represent the starting pattern using a list of numbers, where `1` represents lights switched on and `0` represents lights switched off. The list `lights` represents the starting pattern in this example.

    lights = [0, 0, 0, 1, 0, 1, 1, 1, 0]

The example puzzle can be solved by flipping the switches in sqare 0, 3, and 4. If you play with the example a bit, you will notice two important properties:

1. You don't need to flip a switch more than once, and
2. The order of flipping doesn't matter.

Therefore, we can represent the solution to our example puzzle as a list of numbers similar to the starting pattern. The meaning of `0` and `1` is a bit different: `1` means that a switch is flipped and `0` means that it is *not* flipped.

    solution = [1, 0, 0, 1, 1, 0, 0, 0, 0]

<table style="float:left;margin-right:20px">
<tr><td style="background-color:blue">&nbsp;</td><td style="background-color:blue">&nbsp;</td><td style="background-color:blue">&nbsp;</td></tr>
<tr><td style="background-color:yellow">&nbsp;</td><td style="background-color:blue">&nbsp;</td><td style="background-color:yellow">&nbsp;</td></tr>
<tr><td style="background-color:yellow">&nbsp;</td><td style="background-color:yellow">&nbsp;</td><td style="background-color:blue">&nbsp;</td></tr>
</table>

<table style="float:left;margin-right:20px">
<tr><td style="background-color:yellow">&nbsp;</td><td style="background-color:yellow">&nbsp;</td><td style="background-color:blue">&nbsp;</td></tr>
<tr><td style="background-color:blue">&nbsp;</td><td style="background-color:blue">&nbsp;</td><td style="background-color:yellow">&nbsp;</td></tr>
<tr><td style="background-color:yellow">&nbsp;</td><td style="background-color:yellow">&nbsp;</td><td style="background-color:blue">&nbsp;</td></tr>
</table>

<table style="float:left;margin-right:20px">
<tr><td style="background-color:blue">&nbsp;</td><td style="background-color:blue">&nbsp;</td><td style="background-color:blue">&nbsp;</td></tr>
<tr><td style="background-color:blue">&nbsp;</td><td style="background-color:blue">&nbsp;</td><td style="background-color:blue">&nbsp;</td></tr>
<tr><td style="background-color:blue">&nbsp;</td><td style="background-color:blue">&nbsp;</td><td style="background-color:blue">&nbsp;</td></tr>
</table>

Let's try to solve a "Lights Out" puzzle with the help of Grover's algorithm. Take care that you will need a more complicated oracle than before.

Tip: You will need four registers, one for the input, one for the possible solutions, one for the oracle, and one for the ancillary qubits.

Please implement your circuit within a maximum of 28 qubits.

In [ ]:
import math
from qiskit import Aer
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

from qiskit.tools.visualization import plot_histogram

In [ ]:
import qiskit.tools.jupyter
print("This notebook was created and tested with the following version numbers")
%qiskit_version_table